# Create Project CERES Deliverables for Tennessee farm news (agrtfn)

 - OCR TIFF with Tesseract and output PDF into new directory as 001.pdf, 002.pdf, 00n.pdf

Will separated PDFs into directories based on the publication date after all PDFs have been created. The work to identify the publication date and therefore the correct directory for the deliverable will be done concurrent to PDF construction.

In [5]:
# importing and options
import time
from pathlib import Path

import pandas as pd
from ipywidgets import IntProgress, Label, VBox
from IPython.display import display

# == display 95% width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# set pandas option to display wider column info, use -1 to expand to largest necessary width
pd.set_option('max_colwidth', -1)

## Batch process directories

In [6]:
# create class for processing individual volumes
class Agrtfn:
    def __init__(self, directory_path):
        self.directory_path = Path(directory_path)
        # self.number_of_pages = int(df_tn_farm_and_home_science[df_tn_farm_and_home_science["old_directory_name"].str.contains(directory_path.name)]['pages'].to_string(index=False))
        
    def get_image_paths_list(self):
        
        image_paths_list = sorted(self.directory_path.glob('*.tif'))
        
        # delete all macOS index files that start with '.'
        deleted_dot_files = False
        for image_path in image_paths_list:
            if image_path.name.startswith('.'):
                image_path.unlink()  # delete
                deleted_dot_files = True
        if deleted_dot_files:  # re-create the list
            image_paths_list = sorted(self.directory_path.glob('*.tif'))
        
        self.image_paths_list = image_paths_list
        self.number_of_images = len(self.image_paths_list)
        
        # if self.number_of_images != self.number_of_pages:
        #     print(f'# of images != # of pages described in metadata')
        #     print(f'images: {self.number_of_images}')
        #     print(f'pages: {self.number_of_pages}')
        #     return None
        
        return self.image_paths_list

    
    def ocr(self):
        # will convert TIFF into OCRed PDF with Tesseract
        # PDFs will be named using a 3-digit index
        
        # progress bar
        progress_label = Label('Images being processed')
        progress_bar = IntProgress(min=0, max=self.number_of_images)
        progress_widget = VBox([progress_label, progress_bar])
        display(progress_widget)
        
        for index, image_path in enumerate(self.image_paths_list, start=1):
            
            # update progress bar value
            progress_label.value = f'Processing image: {image_path.name} . . . {index}/{self.number_of_images}'
            
            # NO extension at the end for Tesseract processing
            pdf_output_path = self.directory_path.joinpath(f'{str(index).zfill(3)}')

            # OCR with Tesseract
            !tesseract {str(image_path)} {str(pdf_output_path)} pdf 2>/dev/null
            
            # set output path with extension at end
            pdf_output_path = self.directory_path.joinpath(f'{str(index).zfill(3)}.pdf')
            
            # update progress bar value
            progress_bar.value = index
            
        self.pdf_paths_list = list(self.directory_path.glob('*.pdf'))
        self.number_of_pdfs = len(self.pdf_paths_list)
        
        if self.number_of_pdfs != self.number_of_images:
            print(f'# of PDFs != # of images to OCR')
            print(f'PDFs: {self.number_of_pdfs}')
            print(f'images: {self.number_of_images}')
            return None
        else:
            return self.pdf_paths_list
            
        
#     def process(self):
        
#         # progress bar
#         progress_label = Label('Images being processed')
#         progress_bar = IntProgress(min=0, max=self.number_of_images)
#         progress_widget = VBox([progress_label, progress_bar])
#         display(progress_widget)
        
#         # get output directory name
#         output_directory_name = df_tn_farm_and_home_science[df_tn_farm_and_home_science["old_directory_name"].str.contains(directory_path.name)]['new_directory_name'].to_string(index=False)
#         # strip spaces from output_directory_name
#         self.output_directory_name = output_directory_name.strip()
#         # set output directory path and make directory
#         self.output_directory_path = batch_output_directory_path.joinpath(self.output_directory_name)
#         self.output_directory_path.mkdir()
        
#         # for each *.tif in self.directory_path
#         for index, image_path in enumerate(self.image_paths_list, start=1):
            
#             # update progress bar label
#             progress_label.value = f'Processing image: {image_path.name} . . . {index}/{self.number_of_images}'
            
#             # NO extension at the end for Tesseract processing
#             pdf_output_path = self.output_directory_path.joinpath(f'{str(index).zfill(3)}')
            
#             # OCR with Tesseract
#             !tesseract {str(image_path)} {str(pdf_output_path)} pdf 2>/dev/null
            
#             # set output path with extension at end
#             pdf_output_path = self.output_directory_path.joinpath(f'{str(index).zfill(3)}.pdf')
            
#             # update progress bar value
#             progress_bar.value = index
            
#         self.pdf_paths_list = list(self.output_directory_path.glob('*.pdf'))
#         self.number_of_pdfs = len(self.pdf_paths_list)
        
#         if self.number_of_pdfs != self.number_of_images:
#             print(f'# of PDFs != # of images to OCR')
#             print(f'PDFs: {self.number_of_pdfs}')
#             print(f'images: {self.number_of_images}')
#             return None
#         else:
#             return self.pdf_paths_list

In [9]:
# Set batch_input and batch_output directories

# set project identifier and root directory path
project_identifier = 'agrtfn'
root_directory_path = Path('/Volumes/fluffy/ProjectCeres/00_for_CRL/')

# set batch_input and batch_output directory paths from root and project identifier
batch_input_directory_path = root_directory_path.joinpath(project_identifier)
batch_output_directory_path = root_directory_path.joinpath(f'{project_identifier}_batch_output')
batch_output_directory_path.mkdir()

# create batch_input directory paths list
batch_input_directory_paths_list = sorted([x for x in batch_input_directory_path.iterdir() if x.is_dir()])
number_of_input_dirs = len(batch_input_directory_paths_list)

print(f'batch_input directory: {batch_input_directory_path}')
print(f'\t{number_of_input_dirs} directories to batch process\n')
print(f'batch_output directory: {batch_output_directory_path}')

batch_input directory: /Volumes/fluffy/ProjectCeres/00_for_CRL/agrtfn
	3451 directories to batch process

batch_output directory: /Volumes/fluffy/ProjectCeres/00_for_CRL/agrtfn_batch_output


In [10]:
print(int(number_of_input_dirs*.5))

1725


In [12]:
# process the first half of the paths list
# input_directory_paths_list = batch_input_directory_paths_list[:1726]
# process the second half of the paths list
input_directory_paths_list = batch_input_directory_paths_list[1726:]

number_of_input_dirs = len(input_directory_paths_list)
number_of_input_dirs

1725

In [ ]:
# start batch process

# progress bar
progress_label = Label('Directories being processed')
progress_bar = IntProgress(min=0, max=number_of_input_dirs)
progress_widget = VBox([progress_label, progress_bar])
display(progress_widget)

for index, directory_path in enumerate(input_directory_paths_list, start=1):
    
    # update progress bar label
    progress_label.value = f'Processing directory: {directory_path.name} . . . {index}/{number_of_input_dirs}'
    
    # load directory path as class & process
    issue = Agrtfn(directory_path)
    issue.get_image_paths_list()
    issue.ocr()
    
    #update progress bar value
    progress_bar.value = index

print(f'Processed {index} directories')